# Machine Learning pra Previsão de Evasão Escolar

Neste trabalho será feita uma análise exploratória, limpeza de dados e teste de diversos algoritmos de classificação para prever a evasão escolar.  A acurácia mínima de 90% será buscada.

In [1]:
# Importação dos pacotes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from category_encoders import JamesSteinEncoder
from sklearn.model_selection import train_test_split
import imblearn
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from scipy.sparse import csr_matrix
from scipy import sparse
import pickle
from imblearn.over_sampling import SMOTE

In [2]:
print(imblearn.__version__)

0.8.0


## Limpeza e transformação dos dados.
Observar, limpar e transformar os dados.
Essa primeira limpeza deve ser feita para que quando o algoritmo receber as novas informações, ele receba apenas as colunas necessárias com os dados corretos.

In [3]:
# Carrega o arquivo em um dataset.  Apaga a coluna que vem com o dataset.
tecnicoJoinville = pd.read_csv("TECNICOS_Joinville5.csv", encoding = 'latin-1', low_memory = False)
tecnicoJoinville.drop(["Unnamed: 0"], axis = 1, inplace = True)

In [4]:
# Observando as 5 primeiras linhas do dataset
tecnicoJoinville.head()

,produto_educacao,carga_horaria,turno,gratuidade_turma,tipo,online,bairro,cidade,genero,escolaridade,situacao_ocupacional,situacao_matricula,IDADE,empresa
0,AUTOMACAO,1200,Tarde,Mista,PF,Nao,Aventureiro,Joinville,M,Medio_C,Empregado,Concluinte,21,Schulz
1,FABRICACAO,1600,Noite,Mista,PF,Nao,Jardim_Paraiso,Joinville,M,Medio_C,Empregado,Concluinte,18,N_D
2,LOGISTICA,1300,Noite,Mista,PF,Nao,ForaJoinville,SaoFrancisco_Sul,M,Medio_C,Empregado,Concluinte,18,SENAI/SC
3,AUTOMACAO,1600,Tarde,Mista,PF,Nao,Aventureiro,Joinville,M,Medio_C,Desempregado,Concluinte,16,N_D
4,MECANICA,1620,Tarde,Mista,PF,Nao,Ulysses_Guimaraes,Joinville,M,Medio_C,Empregado,Concluinte,17,Schulz


In [5]:
tecnicoJoinville.shape

(1926, 14)

In [6]:
# Verificando se o dataset está desbalanceado
tecnicoJoinville['situacao_matricula'].value_counts()

Desistente    1178
Concluinte     748
Name: situacao_matricula, dtype: int64

A variável target está desbalanceada. Para que o algoritmo aprenda corretamente é necessário que o dataset esteja balanceado.

## Teste dos algoritmos de classificação

#### Antes de iniciar os testes para escolher o melhor algoritmo, é necessário transformar as variáveis categóricas em variáveis numéricas.
#### Regressão Logística utilizando Ordinal Encoder

In [7]:
# Criando uma lista de nomes dos algoritmos que serão testados. 
# Isso vai permitir criar um dataframe com os resultados de acurácia no final dos testes.
Nomes = ["Regressao", "Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]
Acuracia = []

In [8]:
# Pegando apenas os valores, sem o cabeçalho, em formato de array
data = tecnicoJoinville.values

In [9]:
# Visualizando os dados
data

array([['AUTOMACAO', '1200', 'Tarde', ..., 'Concluinte', 21, 'Schulz'],
       ['FABRICACAO', '1600', 'Noite', ..., 'Concluinte', 18, 'N_D'],
       ['LOGISTICA', '1300', 'Noite', ..., 'Concluinte', 18, 'SENAI/SC'],
       ...,
       ['DESENVOLVIMENTO', '1000', 'Noite', ..., 'Desistente', 20, 'N_D'],
       ['PLASTICO', '1250', 'Noite', ..., 'Desistente', 27, 'N_D'],
       ['PLASTICO', '1250', 'Manha', ..., 'Desistente', 27, 'LINKPLAS']],
      dtype=object)

In [10]:
# Separando a variável target do dataset:
y = data[:,11]

In [11]:
# Tamanho da variável
y.shape

(1926,)

In [12]:
# Pegando todas as outras colunas
X = data[:,[0,1,2,3,4,5,6,7,8,9,10,12,13]]

In [13]:
# Tamanho do array resultante.
X.shape

(1926, 13)

In [14]:
# Visualizando o array
X

array([['AUTOMACAO', '1200', 'Tarde', ..., 'Empregado', 21, 'Schulz'],
       ['FABRICACAO', '1600', 'Noite', ..., 'Empregado', 18, 'N_D'],
       ['LOGISTICA', '1300', 'Noite', ..., 'Empregado', 18, 'SENAI/SC'],
       ...,
       ['DESENVOLVIMENTO', '1000', 'Noite', ..., 'Empregado', 20, 'N_D'],
       ['PLASTICO', '1250', 'Noite', ..., 'Empregado', 27, 'N_D'],
       ['PLASTICO', '1250', 'Manha', ..., 'Empregado', 27, 'LINKPLAS']],
      dtype=object)

In [15]:
# Visualizando a variável target
y

array(['Concluinte', 'Concluinte', 'Concluinte', ..., 'Desistente',
       'Desistente', 'Desistente'], dtype=object)

In [16]:
# Aplicando Label Encoder aos dados de saída
label_encoder = LabelEncoder()
label_encoder.fit(y)
y = label_encoder.transform(y)

In [17]:
# Visualizando a variável target de treino após a aplicação do Label Encoder
y

array([0, 0, 0, ..., 1, 1, 1])

### James Stein Encoder

In [18]:
js_encoder = JamesSteinEncoder()
js_encoder.fit_transform(X=X, y=y)
X = js_encoder.transform(X)

D:\ANACONDA\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [19]:
X[np.isfinite(X) == False] = 0

In [20]:
scaler = StandardScaler(with_mean=False) 
X = X.astype(np.float64)
X = scaler.fit_transform(X)
X = scaler.fit_transform(X)
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)

In [21]:
# Separando em conjunto de treino e teste
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y, test_size=0.33, random_state = 0)

In [22]:
#Definindo o modelo
modelo5 = LogisticRegression(solver='saga', max_iter=10000)

In [23]:
modelo5.fit(X_train1, y_train1)
# Previsões nos dados de teste
yhat = modelo5.predict(X_test1)
# Avalia a previsão
accuracy = accuracy_score(y_test1, yhat)
print('Accuracy: %.2f' % (accuracy*100))

# Salvando o resultado da regressão logística com Ordinal Encoder
Acuracia.append(accuracy*100)  ##################################  1

Accuracy: 85.86


## Utilizar o K Nearest Neighbors com James Stein Encoder

In [24]:
# Cria o modelo
modeloK =  KNeighborsClassifier(30)

In [25]:
# Treina o modelo
modeloK.fit(X_train1, y_train1)
# Fazendo previsões
yhat1 = modeloK.predict(X_test1)
score = modeloK.score(X_test1, y_test1)
# Avaliando as previsões
accuracy = accuracy_score(y_test1, yhat1)
print(score*100)

Acuracia.append(score*100) ######################## 2

87.40359897172236


## Utilizar o SVC com  James Stein Encoder

In [26]:
# Cria o modelo
modeloSVC = SVC(kernel="linear", C=0.025)

# Treina o modelo
modeloSVC.fit(X_train1, y_train1)
# Fazendo previsões
score = modeloSVC.score(X_test1, y_test1)

# Avaliando as previsões
print(score*100)

Acuracia.append(score*100)   ######################## 3

87.40359897172236


## Utilizar o SVC com outros parâmetros, James Stein Encoder

In [27]:
# Cria o modelo
modeloSVC = SVC(gamma=2, C=1)

# Treina o modelo
modeloSVC.fit(X_train1, y_train1)
# Fazendo previsões
score = modeloSVC.score(X_test1, y_test1)

# Avaliando as previsões
print(score*100)

Acuracia.append(score*100)   ######################## 4

85.47557840616966


## Utilizar o classificador Gaussiano, James Stein Encoder

In [28]:
# Cria o modelo
modeloGaus = GaussianProcessClassifier(1.0 * RBF(1.0))

# Treina o modelo
modeloGaus.fit(X_train1, y_train1)
# Fazendo previsões
score = modeloGaus.score(X_test1, y_test1)

# Avaliando as previsões
print(score*100)

    
Acuracia.append(score*100) ######################## 5

89.33161953727506


## Utilizar o Decision Tree, James Stein Encoder

In [29]:
# Cria o modelo
modeloDT = DecisionTreeClassifier(max_depth=5)

# Treina o modelo
modeloDT.fit(X_train1, y_train1)
# Fazendo previsões
score = modeloDT.score(X_test1, y_test1)

# Avaliando as previsões
print(score*100)

Acuracia.append(score*100)  ######################## 6

83.9331619537275


## Utilizar o Random Forest, James Stein Encoder

In [30]:
# Cria o modelo
modeloRF = RandomForestClassifier(max_depth=50, n_estimators=100, max_features=13)

# Treina o modelo
modeloRF.fit(X_train1, y_train1)
# Fazendo previsões
score = modeloRF.score(X_test1, y_test1)

# Avaliando as previsões
print(score*100)
previsoes = modeloRF.predict(X_test1)
Acuracia.append(score*100)  ######################## 7

89.33161953727506


## Utilizar uma rede neural, James Stein Encoder


In [31]:

# Cria o modelo
modeloMLP = MLPClassifier(alpha=2, max_iter=2000)

# Treina o modelo
modeloMLP.fit(X_train1, y_train1)
# Fazendo previsões
score = modeloMLP.score(X_test1, y_test1)

# Avaliando as previsões
print(score*100)

Acuracia.append(score*100)  ######################## 8

84.31876606683805


## Utilizar o classificador ADA Boost, James Stein Encoder

In [32]:
# Cria o modelo
modeloABC =  AdaBoostClassifier()

# Treina o modelo
modeloABC.fit(X_train1, y_train1)
# Fazendo previsões
score = modeloABC.score(X_test1, y_test1)

# Avaliando as previsões
print(score*100)

Acuracia.append(score*100)  ######################## 9

87.91773778920309


## Utilizar o classificador Naive Bayes, James Stein Encoder

In [33]:
# Cria o modelo
modeloGNB =  GaussianNB()

# Treina o modelo
modeloGNB.fit(X_train1, y_train1)
# Fazendo previsões
score = modeloGNB.score(X_test1, y_test1)

# Avaliando as previsões
print(score*100)

Acuracia.append(score*100)  ######################## 10

80.20565552699229


## Utilizar o classificador Quadratic Discriminant Analysis, James Stein Encoder

In [34]:
# Cria o modelo
modeloQDA =  QuadraticDiscriminantAnalysis()

# Treina o modelo
modeloQDA.fit(X_train1, y_train1)
# Fazendo previsões
score = modeloQDA.score(X_test1, y_test1)

# Avaliando as previsões
print(score*100)

Acuracia.append(score*100) ######################## 11

81.61953727506427


In [35]:
Acuracia

[85.8611825192802,
 87.40359897172236,
 87.40359897172236,
 85.47557840616966,
 89.33161953727506,
 83.9331619537275,
 89.33161953727506,
 84.31876606683805,
 87.91773778920309,
 80.20565552699229,
 81.61953727506427]

In [36]:
#Resultados = pd.DataFrame(Nomes, Acuracia)
Resultados = pd.DataFrame(list(zip(Nomes, Acuracia)),columns =['Nomes', 'Acuracia'])

In [37]:
Resultados

,Nomes,Acuracia
0,Regressao,85.861183
1,Nearest Neighbors,87.403599
2,Linear SVM,87.403599
3,RBF SVM,85.475578
4,Gaussian Process,89.331620
5,Decision Tree,83.933162
6,Random Forest,89.331620
7,Neural Net,84.318766
8,AdaBoost,87.917738
9,Naive Bayes,80.205656


In [38]:

print('Modelo escolhido: %.2f' % (Resultados['Acuracia'].max()))

Modelo escolhido: 89.33


In [39]:
modeloRF

RandomForestClassifier(max_depth=50, max_features=13)

In [40]:
# Desistente ==> 1
# Concluinte ==> 0
previsoes = modeloRF.predict(X_test1)
print(previsoes)

[0 1 0 1 1 0 1 0 1 1 1 0 0 1 1 1 1 0 0 0 0 1 1 1 1 1 0 1 1 0 1 0 0 1 1 1 1
 1 1 1 1 0 1 0 0 1 0 0 1 1 1 1 0 0 1 1 1 1 1 0 0 1 1 0 0 1 1 1 0 0 1 0 0 1
 0 1 0 0 1 0 1 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 1 0
 0 1 1 0 1 0 0 1 0 1 1 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 1
 1 1 1 0 0 1 0 0 1 0 1 1 0 1 0 1 1 0 1 1 1 0 0 1 0 0 1 1 0 1 0 1 1 1 1 0 1
 1 0 0 1 0 1 0 0 1 1 0 0 0 0 1 0 0 0 1 1 1 0 0 0 0 1 1 0 0 1 0 1 0 0 1 0 0
 1 1 1 1 0 0 1 1 0 1 1 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 1 0 0 0
 0 1 1 1 1 0 1 1 1 0 1 0 0 1 1 1 1 0 1 1 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 0 1 1 1 0 0 1 0 1 0 0 0 0 0 0 1 0 0 1 1 1 1 0 1 1 1 0 0 1 0 1
 1 1 0 0 1 1 0 0 1 0 0 0 1 1 1 0 1 0 0 1 1 1 1 1 0 0 0 0 0 1 1 0 1 0 0 0 1
 0 1 1 0 1 0 0 0 0 1 0 1 0 1 0 0 1 0 1 1 1 1 0 1 1 1 1 0 1 0 1 1 1 0 1 0 1
 0 1 0 0 0 1 0 1 0 0 1 1 1 0 1 1 0 0 0 1 1 0 0 0 1 0 1 1 1 1 0 1 1 0 0 1 0
 0 1 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 1 0 1 1 1 1 0 1 0 0 1 0 1 1 0 1 0 1 1
 1 0 0 0 1 0 1 0 1 1 1 1 

In [41]:
len(previsoes)

778

In [42]:
respFinal = []
for i in range(0,len(previsoes)):
    if previsoes[i] == 1:
        respFinal.append("Desistente")
    else:
        respFinal.append("Concluinte")
respFinal
    

['Concluinte',
 'Desistente',
 'Concluinte',
 'Desistente',
 'Desistente',
 'Concluinte',
 'Desistente',
 'Concluinte',
 'Desistente',
 'Desistente',
 'Desistente',
 'Concluinte',
 'Concluinte',
 'Desistente',
 'Desistente',
 'Desistente',
 'Desistente',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Concluinte',
 'Desistente',
 'Desistente',
 'Desistente',
 'Desistente',
 'Desistente',
 'Concluinte',
 'Desistente',
 'Desistente',
 'Concluinte',
 'Desistente',
 'Concluinte',
 'Concluinte',
 'Desistente',
 'Desistente',
 'Desistente',
 'Desistente',
 'Desistente',
 'Desistente',
 'Desistente',
 'Desistente',
 'Concluinte',
 'Desistente',
 'Concluinte',
 'Concluinte',
 'Desistente',
 'Concluinte',
 'Concluinte',
 'Desistente',
 'Desistente',
 'Desistente',
 'Desistente',
 'Concluinte',
 'Concluinte',
 'Desistente',
 'Desistente',
 'Desistente',
 'Desistente',
 'Desistente',
 'Concluinte',
 'Concluinte',
 'Desistente',
 'Desistente',
 'Concluinte',
 'Concluinte',
 'Desistente',
 'Desisten

In [43]:
# save the model to disk
filename = 'modeloRF.sav'
pickle.dump(modeloRF, open(filename, 'wb'))